# Data exploration

## Notebook setup

In [ ]:
import pickle
import os

In [ ]:
import numpy as np
import plotly.express as px

In [ ]:
DATA_DIR = os.path.join('..', 'data')
NOTEBOOK1_OUTPUT_DIR = os.path.join('output', '1_data_fetching')

CLASSES_FILE = os.path.join(NOTEBOOK1_OUTPUT_DIR, 'top_classes.pkl')

## Loading the data from the previous notebook

In [ ]:
with open(CLASSES_FILE, 'rb') as f:
    top_classes = pickle.load(f)

## Displaying general information

In [ ]:
print("Top 25 classes based on classrank")
print("-" * 35)
print('\n'.join([f"{c.name} - score: {c.classrank_score}" for c in top_classes[:25]]))

In [ ]:
# Histogram
x = [len(c.top_entities) for c in top_classes]
labels = {
    "x": "Number of class instances"
}

fig = px.histogram(x=x, nbins=10, template='plotly_white', labels=labels)
fig.update_layout(bargap=0.02)
fig.show()

In [ ]:
fig.write_html("./sample.html")

In [ ]:
fig.write_image("./sample.svg")

## Querying the diff data

In [1]:
!pip install pymongo

In [2]:
import getpass
import os

def env_or_callback(env_name, callback, *args):
    return os.getenv(env_name) if os.getenv(env_name) is not None else callback(*args)

In [3]:
from pymongo import MongoClient
import pymongo

MONGO_USERNAME = env_or_callback("MONGO_USERNAME", input, "Mongo username: ")
MONGO_PASSWORD = env_or_callback("MONGO_PASSWORD", getpass.getpass, "Mongo password: ")
MONGO_URL = env_or_callback("MONGO_URL", input, "Mongo url: ")
MONGO_DATABASE = "wd_diff"

def get_database():
    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_URL}/{MONGO_DATABASE}"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(CONNECTION_STRING)

    # Create the database for our example (we will use the same database throughout the tutorial
    return client[MONGO_DATABASE]

In [4]:
db = get_database()
wd_entities = db.wd_entities

In [ ]:
wd_entities.drop_indexes()

In [9]:
pprint.pprint(list(wd_entities.list_indexes()))

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]),
 SON([('v', 2), ('key', SON([('entity_id', -1)])), ('name', 'entity_id_-1')]),
 {'key': SON([('entity_json.claims.P31.mainsnak.davalue.value.id', -1)]),
  'name': 'entity_json.claims.P31.mainsnak.davalue.value.id_-1',
  'sparse': True,
  'v': 2},
 {'key': SON([('revisions.entity_diff.op', -1)]),
  'name': 'revisions.entity_diff.op_-1',
  'v': 2}]


In [ ]:
wd_entities.create_index([("entity_id", pymongo.DESCENDING)])
#wd_entities.create_index([("entity_json.claims.P31", pymongo.DESCENDING)])
wd_entities.create_index([("entity_json.claims.P31.mainsnak.davalue.value.id", pymongo.DESCENDING)], sparse=True)
#wd_entities.create_index([("revisions", pymongo.DESCENDING)])
wd_entities.create_index([("revisions.entity_diff.op", pymongo.DESCENDING)])
#wd_entities.create_index([("entity_json.revisions", pymongo.DESCENDING)])

In [7]:
import pprint

def test(collection):
    return collection.find_one({"entity_id": "Q25107362"})

def get_doc_count(collection):
    return collection.find_one({"entity_id": "Q25107362"})

def get_num_instances_of(collection, q_id):
    return collection.count_documents({"entity_json.claims.P31.mainsnak.datavalue.value.id": q_id})

def get_instances_of(collection, q_id):
    return collection.find({"entity_json.claims.P31.mainsnak.datavalue.value.id": q_id})

def get_instances_of_c(collection, ids):
    return collection.find({"entity_id": {"$in": ids}})

def get_number_of_editions(collection, q_id):
    pipeline = [
        {"$match": {"entity_json.claims.P31": {"$elemMatch": {"mainsnak.datavalue.value.id": q_id}}}},
        {"$group": {"_id": "null", "num_revisions": { "$sum": { "$size": "$revisions" }}}}
    ]
    return collection.aggregate(pipeline)

def get_number_of_operations(collection, q_id):
    pipeline = [
        {"$match": {"entity_json.claims.P31.mainsnak.datavalue.value.id": q_id}},
        {"$unwind": "$revisions"},
        {"$unwind": "$revisions.entity_diff"},
        {
            "$group": 
            {
                "_id": "$revisions.entity_diff.op",
                "count": {"$sum" : 1}
            }
        }
    ]
    return collection.aggregate(pipeline)

def get_number_of_operations_b(collection, ids):
    pipeline = [
        {"$match": {"entity_id": {"$in": ids}}},
        {"$unwind": "$revisions"},
        {"$unwind": "$revisions.entity_diff"},
        {
            "$group": 
            {
                "_id": "$revisions.entity_diff.op",
                "count": {"$sum" : 1}
            }
        }
    ]
    return collection.aggregate(pipeline)


In [8]:
pprint.pprint(get_num_instances_of(wd_entities, 'Q5'))

870358


In [14]:
import random

def get_random_ids(n=500000):
    ids = ['Q25107362']
    for i in range(n):
        ids.append(f"Q{random.randint(10000000, 100000000)}")
    return ids

In [33]:
HUMAN_QID = "Q5"

In [ ]:
pprint.pprint(get_instances_of(wd_entities, HUMAN_QID))

In [34]:
pprint.pprint(get_instances_of_b(wd_entities, HUMAN_QID))

In [31]:
pprint.pprint(len(list(get_instances_of_c(wd_entities, ids))))

10836


In [38]:
pprint.pprint(list(get_number_of_operations(wd_entities, HUMAN_QID)))

[]


In [30]:
pprint.pprint(list(get_number_of_operations_b(wd_entities, ids)))

[{'_id': 'replace', 'count': 48931},
 {'_id': 'remove', 'count': 12695},
 {'_id': 'add', 'count': 327815}]


In [40]:
%%time

# TODO: replace classes by real data
classes = [
    ('human', 'Q5'),
    ('sovereign state', 'Q3624078'),
    ('taxon', 'Q16521'),
    ('country', 'Q6256'),
    ('city', 'Q515')
]

results = []

for name, c_id in classes:
    res = get_number_of_operations(wd_entities, c_id)
    results.append((name, list(res)))

pprint.pprint(results)

[('human',
  [{'_id': 'add', 'count': 33316069},
   {'_id': 'remove', 'count': 751807},
   {'_id': 'replace', 'count': 3648424}]),
 ('sovereign state',
  [{'_id': 'replace', 'count': 293},
   {'_id': 'remove', 'count': 89},
   {'_id': 'add', 'count': 573}]),
 ('taxon',
  [{'_id': 'add', 'count': 2204879},
   {'_id': 'remove', 'count': 48028},
   {'_id': 'replace', 'count': 460201}]),
 ('country', [{'_id': 'add', 'count': 10}, {'_id': 'replace', 'count': 1}]),
 ('city',
  [{'_id': 'add', 'count': 8843},
   {'_id': 'remove', 'count': 512},
   {'_id': 'replace', 'count': 1587}])]
CPU times: user 523 ms, sys: 172 ms, total: 695 ms
Wall time: 14min 34s


In [42]:
%%time

# TODO: replace classes by real data
classes = [
    ('human', 'Q5'),
    ('sovereign state', 'Q3624078'),
    ('taxon', 'Q16521'),
    ('country', 'Q6256'),
    ('city', 'Q515')
]
results = []

for name, c_id in classes:
    res = get_number_of_operations_b(wd_entities, get_random_ids())
    results.append((name, list(res)))

pprint.pprint(results)

[('human',
  [{'_id': 'remove', 'count': 15784},
   {'_id': 'add', 'count': 367698},
   {'_id': 'replace', 'count': 56332}]),
 ('sovereign state',
  [{'_id': 'replace', 'count': 59264},
   {'_id': 'add', 'count': 370463},
   {'_id': 'remove', 'count': 15294}]),
 ('taxon',
  [{'_id': 'remove', 'count': 15398},
   {'_id': 'add', 'count': 372588},
   {'_id': 'replace', 'count': 64352}]),
 ('country',
  [{'_id': 'replace', 'count': 53430},
   {'_id': 'add', 'count': 355649},
   {'_id': 'remove', 'count': 14392}]),
 ('city',
  [{'_id': 'add', 'count': 362925},
   {'_id': 'replace', 'count': 53871},
   {'_id': 'remove', 'count': 14004}])]
CPU times: user 3.4 s, sys: 236 ms, total: 3.63 s
Wall time: 2min 51s


In [ ]:
%%time

res = get_number_of_editions(wd_entities, HUMAN_QID)
pprint.pprint(list(res))

In [ ]:
pprint.pprint(list(res))

In [ ]:
%%time

print(len(get_instances_of(wd_entities, HUMAN_QID)))

In [92]:
def get_properties_edited(collection, q_id):
    pipeline = [
        {"$match": {"entity_json.claims.P31.mainsnak.datavalue.value.id": q_id}},
        {"$unwind": "$revisions"},
        {"$unwind": "$revisions.entity_diff"},
        {"$match": {"revisions.entity_diff.path": {"$regex": "\/claims\/P([0-9]*)$"}}},
        {"$addFields": {"prop": {"$regexFind": {"input": "$revisions.entity_diff.path", "regex": "P([0-9]*)"}}}},
        {
            "$group": 
            {
                "_id": {"prop": "$prop.match", "op": "$revisions.entity_diff.op"},
                "count": {"$sum" : 1}
            }
        },
        {"$sort": {"count": -1}}
    ]
    return collection.aggregate(pipeline)

def get_properties_edited_b_batch(collection, ids):
    pipeline = [
        {"$match": {"entity_id": {"$in": ids}}},
        {"$unwind": "$revisions"},
        {"$unwind": "$revisions.entity_diff"},
        {"$match": {"revisions.entity_diff.path": {"$regex": "\/claims\/P([0-9]*)$"}}},
        {"$addFields": {"prop": {"$regexFind": {"input": "$revisions.entity_diff.path", "regex": "P([0-9]*)"}}}},
        {
            "$group": 
            {
                "_id": {"prop": "$prop.match", "op": "$revisions.entity_diff.op"},
                "count": {"$sum" : 1}
            }
        },
        {"$sort": {"count": -1}}
    ]
    return collection.aggregate(pipeline)

def get_properties_edited_b(collection, ids):
    batch_size = 300000
    num_batchs = math.ceil(len(ids) / batch_size)
    current_results = None
    for i in num_batchs:
        start = i * batch_size
        end = (i+1) * batch_size
        batch = ids[start:end]
        res = list(get_properties_edited_b_batch(collection, batch))
        if current_results is None:
            current_results = res
            continue
        
        _ids = [r['_id'] for r in res]
        new_elements = []
        for el1 in res:
            match = False
            for el2 in current_results:
                if el1['_id'] == el2['_id']:
                    el2['count'] += el1['count']
                    match = True
                    break
            if not match:
                new_elements.append(el1)
        current_result += new_elements
    

In [85]:
# 20 most popular changes
pprint.pprint(list(get_properties_edited(wd_entities,HUMAN_QID))[:20])

[{'_id': {'op': 'add', 'prop': 'P31'}, 'count': 788278},
 {'_id': {'op': 'add', 'prop': 'P21'}, 'count': 765414},
 {'_id': {'op': 'add', 'prop': 'P569'}, 'count': 717822},
 {'_id': {'op': 'add', 'prop': 'P2671'}, 'count': 696782},
 {'_id': {'op': 'add', 'prop': 'P106'}, 'count': 681095},
 {'_id': {'op': 'add', 'prop': 'P735'}, 'count': 494242},
 {'_id': {'op': 'add', 'prop': 'P19'}, 'count': 446756},
 {'_id': {'op': 'add', 'prop': 'P27'}, 'count': 433739},
 {'_id': {'op': 'add', 'prop': 'P734'}, 'count': 283292},
 {'_id': {'op': 'add', 'prop': 'P570'}, 'count': 275710},
 {'_id': {'op': 'add', 'prop': 'P1412'}, 'count': 202813},
 {'_id': {'op': 'add', 'prop': 'P641'}, 'count': 193610},
 {'_id': {'op': 'add', 'prop': 'P69'}, 'count': 184383},
 {'_id': {'op': 'add', 'prop': 'P214'}, 'count': 176446},
 {'_id': {'op': 'add', 'prop': 'P7859'}, 'count': 139108},
 {'_id': {'op': 'add', 'prop': 'P18'}, 'count': 135556},
 {'_id': {'op': 'add', 'prop': 'P20'}, 'count': 130132},
 {'_id': {'op': 'a

In [ ]:
# 20 most popular changes
ids = list(set(get_random_ids(9000000)))
pprint.pprint(list(get_properties_edited_b(wd_entities, ids))[:20])

In [ ]:
# groups also by values being edited
def get_properties_edited_detailed(collection, ids):
    pipeline = [
        {"$match": {"entity_id": {"$in": ids}}},
        {"$unwind": "$revisions"},
        {"$unwind": "$revisions.entity_diff"},
        {"$match": {"revisions.entity_diff.path": {"$regex": "\/claims\/P([0-9]*)$"}}},
        {
            "$group": 
            {
                "_id": {"path": "$revisions.entity_diff.path", "op": "$revisions.entity_diff.op", "omfg": "$revisions.entity_diff.value.mainsnak.datavalue"},
                "count": {"$sum" : 1}
            }
        },
        {"$sort": {"count": -1}}
    ]
    return collection.aggregate(pipeline)

In [79]:
def get_properties_edited_deciles(collection, ids, decile_num=0):
    if decile_num < 0 or decile_num > 9:
        print("Decile must be a number between 0 and 9")
        return []
    
    pipeline = [
        {"$match": {"entity_id": {"$in": ids}}},
        {"$project": {
            "revs": {
                "$slice": [
                    "$revisions",
                    # calculate start and end of given decile
                    {"$multiply": [{"$ceil": {"$divide": [{"$size": "$revisions"}, 10]}}, decile_num]}, 
                    {"$multiply": [{"$ceil": {"$divide": [{"$size": "$revisions"}, 10]}}, decile_num+1]}
                ]
            }
        }},
        {"$unwind": "$revs"},
        {"$unwind": "$revs.entity_diff"},
        {"$match": {"revs.entity_diff.op": "remove", "revs.entity_diff.path": {"$regex": "\/claims\/P([0-9]*)$"}}},
        {"$addFields": {"prop": {"$regexFind": {"input": "$revs.entity_diff.path", "regex": "P([0-9]*)"}}}},
        {
            "$group": 
            {
                "_id": {"prop": "$prop.match", "op": "$revs.entity_diff.op"},
                "count": {"$sum" : 1}
            }
        },
        {"$sort": {"count": -1}}
    ]
    return collection.aggregate(pipeline)


In [19]:
ids = get_random_ids(20000)

In [84]:
HUMAN_QID = 'Q5'

pprint.pprint(list(get_properties_edited_deciles(wd_entities, ids, 4))[:20])

[{'_id': ObjectId('6206d58e228b9245ac7292aa'),
  'revs': {'comment': '/* wbsetclaim-create:2||1 */ [[Property:P106]]: '
                      '[[Q2252262]], BOT - Adding 1 claim',
           'entity_diff': {'op': 'add',
                           'path': '/claims/P106',
                           'value': [{'id': 'Q99734855$725805db-9fa7-4741-878b-ee8a35408208',
                                      'mainsnak': {'datavalue': {'type': 'wikibase-entityid',
                                                                 'value': {'entity-type': 'item',
                                                                           'id': 'Q2252262',
                                                                           'numeric-id': 2252262}},
                                                   'hash': '6ec748e648ccf3b5cc038bae33712e25411b0b2f',
                                                   'property': 'P106',
                                                   'snaktype': 'value'},
   

## TODO
- ver si existe una correlación entre las ediciones (numero medio de ediciones, de borrados...) y la importancia de cada clase (valor classrank) -> scatter plot/analisis de correlacion
- obtener clases más "estables"/en las que hay más consenso. p.ej. mirar la media de ediciones por clase, o la media de borrados por clase. etc
- sacar propiedades más añadidas/eliminadas por clase (y global) tanto en general como en cada decil (primer decil, segundo decil...)
- mismo de arriba pero en vez de por deciles a nivel temporal (en el primer dia, la primera semana, el primer mes, el primer año, despúes...)
- sacar propiedades más comunes en cada clase
- ver si podemos detectar vandalismo y quitarlo de las medidas (p.ej. una entidad muy conocido puede tener mucho vandalismo que nos haga pensar que no es estable cuando en realidad igual sí lo es)
